In [ ]:
source("Main.R")
source("Conf.R")
source("Utilities.R")
library("factoextra")
library("maptree")
library(repr)
library("igraph")

In [ ]:
coefsAll <- as.data.frame(read.csv(file.path(projectDir,"MixedEffectLMOutputs",
                                             "ME_LMBetaCoefsALL.csv"), 
                                   row.names = 1),
                          stringsAsFactors = FALSE)

coefsAll <- coefsAll[rownames(coefsAll) %ni% c("Intercept", "n_genes", "mt_frac", "Group Var"),]

pValsAllAdj <- as.data.frame(read.csv(file.path(projectDir,"MixedEffectLMOutputs",
                                                "ME_LMBetaFDRALL.csv"), 
                     row.names = 1), stringsAsFactors = FALSE)

In [ ]:
guideModules = read.csv("./../TextFiles/ME_GuideModules_leiden_6_Modules.csv", row.names = 1)
guideModules$GuideName <- NULL
guideModules$GuideGroup <- paste0("K", guideModules$GuideGroup)
guideModules$GuideGroup <- factor(guideModules$GuideGroup)

In [ ]:
rownames(coefsAll) = sapply(rownames(coefsAll), 
                            function(x){strsplit(as.character(x),"_")[[1]][2]})
rownames(pValsAllAdj) = sapply(rownames(pValsAllAdj), 
                               function(x){strsplit(as.character(x),"_")[[1]][2]})
coefsAll[pValsAllAdj > 0.1 ] = 0

In [ ]:
coefsAllSign = as.data.frame(read.csv(file.path(projectDir,"MixedEffectLMOutputs",
                                                "ME_SignificantBetaCoefs.csv"),
                                      row.names = 1),
                             stringsAsFactors = FALSE)
rownames(coefsAllSign) = sapply(rownames(coefsAllSign),
                                function(x){strsplit(as.character(x),"_")[[1]][2]})
pValsAllAdjSgn <- pValsAllAdj[rownames(coefsAllSign), colnames(coefsAllSign)]
coefsAllSign[pValsAllAdjSgn > 0.1] = 0

In [ ]:
coefsAll$Rfwd2 = coefsAll$Cop1
coefsAll$Tceb1 = coefsAll$Eloc  
coefsAll$Tceb3 = coefsAll$Eloa
coefsAll$Vprbp = coefsAll$Dcaf1  
coefsAll$Tceb2 = coefsAll$Elob

In [ ]:
koGuideNames <- rownames(coefsAll)
length(koGuideNames)

In [ ]:
sgnkoGuideNames <- rownames(coefsAllSign)
length(sgnkoGuideNames)

In [ ]:
availKOs = koGuideNames[koGuideNames %in% colnames(coefsAll)]
length(availKOs)

In [ ]:
coefsAllE3s = coefsAll[availKOs, availKOs]
dim(coefsAllE3s)

In [ ]:
availKOsSgn = sgnkoGuideNames[sgnkoGuideNames %in% colnames(coefsAll)]
length(availKOsSgn)

In [ ]:
realE3s = as.data.frame(read.csv(file.path(projectDir,"ManuscriptFigures","220610_regulators_metadata_E3_Complex.csv")),
                             stringsAsFactors = FALSE)
e3s = unique(realE3s$Symbol_guides)
e3s = e3s[e3s != '']

In [ ]:
length(e3s)

In [ ]:
availE3s = e3s[e3s %in% colnames(coefsAll)]
length(availE3s)

In [ ]:
e3s[e3s %ni% colnames(coefsAll)]

In [ ]:
k = diag(as.matrix(coefsAllE3s))

In [ ]:
length(k[k != 0])

In [ ]:
length(k[k < 0])

In [ ]:
coefsAllE3s = coefsAll[availKOs, availKOs]
options(repr.plot.width=8, repr.plot.height=8)

coefsAllE3s[coefsAllE3s > 0.2] = 0.2
coefsAllE3s[coefsAllE3s < -0.2] = -0.2

xx <- pheatmap(coefsAllE3s, treeheight_col=0, treeheight_row=0, fontsize = 3,
         cluster_rows=FALSE, cluster_cols=FALSE, color=colorRampPalette(c("navy", "white", "red"))(50))

#save_pheatmap_pdf(xx, "../SupplementaryFigures/S_1H.pdf", width=4, height=4)

In [ ]:
coefsAllE3s = coefsAll[e3s, availE3s]

commonNames = rownames(coefsAllE3s)[rownames(coefsAllE3s) %in% colnames(coefsAllE3s)]
for(elem in commonNames){
    coefsAllE3s[elem,elem] = 0
}



dim(coefsAllE3s)

In [ ]:
nonzeroVals = apply(coefsAllE3s,1, function(x){length(x[x!= 0])})
nonzeroVals = nonzeroVals[nonzeroVals > 0]
coefsAllE3s = coefsAllE3s[names(nonzeroVals),]


dim(coefsAllE3s)

In [ ]:
nonzeroEfGenes = apply(coefsAllE3s,2, function(x){length(x[x!= 0])})
nonzeroEfGenes = nonzeroEfGenes[nonzeroEfGenes > 0]
coefsAllE3s = coefsAllE3s[,names(nonzeroEfGenes)]
dim(coefsAllE3s)

In [ ]:
options(repr.plot.width=6, repr.plot.height=8)

annoCols<-list(GeneGroup=c(G0='#A6CEE3',
                           G1='#1F78B4',
                           G2='#B2DF8A',
                           G3='#33A02C', 
                           G4='#FB9A99', 
                           G5='#FDBF6F', 
                           G6='#FF7F00', 
                           G7='#CAB2D6',
                           G8='#6A3D9A', 
                           G9='#FFFF99', 
                           G10="#B5651D" ),
                 GuideGroup=c(K0="#1f77b4",
                              K1="#ff7f0e",
                              K2="#279e68",
                              K3="#d62728", 
                              K4="#aa40fc", 
                              K5="#8c564b"))
   
coefsAllE3s[coefsAllE3s > 0] = 1
coefsAllE3s[coefsAllE3s < 0] = -1

annotDFcol <- guideModules[colnames(coefsAllE3s),]
annotDFrow <- guideModules[rownames(coefsAllE3s),]

annotDFcol$GuideColor = NULL
annotDFrow$GuideColor = NULL

xx <- pheatmap(coefsAllE3s, 
               treeheight_col=0, 
               treeheight_row=0, 
               fontsize = 8,
               clustering_method = "ward.D2",
               cluster_rows=TRUE,
               cluster_cols=TRUE, 
            annotation_col = annotDFcol,
            annotation_row = annotDFrow,
            annotation_colors = annoCols,
               color=colorRampPalette(c("navy", "white", "red"))(50)
        )


save_pheatmap_pdf(xx, "Figure_S3B.pdf", width=7, height=8)

In [ ]:
coefsAllE3s$KOGene = rownames(coefsAllE3s)
coefsAllE3sSgnMelted = melt(coefsAllE3s, id.vars="KOGene")
coefsAllE3sSgnMelted = coefsAllE3sSgnMelted[coefsAllE3sSgnMelted$value != 0,]
colnames(coefsAllE3sSgnMelted) = c("KOGene", "AffectedGene", "Value")

In [ ]:
set.seed(123)

options(repr.plot.width=15, repr.plot.height=15)
df.g <- graph.data.frame(d = coefsAllE3sSgnMelted, directed = TRUE)
#df.g <- simplify(df.g)
V(df.g)$label.cex = 1.7
df.g
E(df.g)$sign = ifelse( coefsAllE3sSgnMelted$Value > 0, 1, -1)
E(df.g)$color <- ifelse( E(df.g)$sign > 0, "pink", "lightblue")

coords <- layout_(df.g, nicely())

plot(df.g, vertex.label = V(df.g)$name, vertex.size=1.5, vertex.label.dist=0.2, layout = coords)


In [ ]:
kkk = colnames(coefsAllE3s)[colnames(coefsAllE3s) %in% rownames(coefsAllE3s)]
tmp = guideModules[kkk,]
tmp=tmp[order(tmp$GuideGroup),]
kkk=rownames(tmp)
kkk

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)

onlyE3s = coefsAll[kkk,kkk]

annotDFcol <- guideModules[colnames(onlyE3s),]
annotDFrow <- guideModules[rownames(onlyE3s),]

annotDFcol$GuideColor = NULL
annotDFrow$GuideColor = NULL

onlyE3s[onlyE3s > 0] = 1
onlyE3s[onlyE3s < 0] = -1

xx <- pheatmap(onlyE3s, 
               treeheight_col=0, 
               treeheight_row=0, 
               fontsize = 12,
               clustering_method = "ward.D2",
               cluster_rows=FALSE,
               cluster_cols=FALSE, 
            annotation_col = annotDFcol,
            annotation_row = annotDFrow,
            annotation_colors = annoCols,
               color=colorRampPalette(c("navy", "white", "red"))(50)
        )

save_pheatmap_pdf(xx, "Figure_S3XX.pdf", width=6, height=4)

In [ ]:
c2 = cluster_edge_betweenness(df.g)
modularity(c2)